In [1]:
###FINAL###
#This section is focused on analysis of the group of diabetic patients, to ascertain if their mean blood glucose values change (improve or 
#worsen) over time and if so, whether this difference is statistically significant or not.

<img src= "InferentialCohortAnalysis1.png" style = "width:1000px; height:300px" />

In [2]:
#IMPORT LIBRARIES

In [9]:
library(dplyr)
library(ggplot2)
library(tidyr)
install.packages("caret"); library(caret)
library(lubridate)
library(purrr)
library(mgcv)
library(rpart)
install.packages("sigr"); library(sigr)
library(tidyverse)
library(tidyselect)
library(tidymodels)
library(utils)
library(base)
library(broom)
library(cluster)
library(ellipsis)
library(graphics)
library(grDevices)
library(highr)
library(lattice)
library(methods)
library(nlme)
library(readr)
library(readxl)
library(stats)
library(stringr)
install.packages("ranger"); library(ranger)
library(tibble)
install.packages("Metrics"); library(Metrics)
install.packages("hms"); library(hms)


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: lattice

Attaching package: 'caret'

The following object is masked from 'package:purrr':

    lift


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: nlme

Attaching package: 'nlme'

The following object is masked from 'package:dplyr':

    collapse

This is mgcv 1.8-28. For overview type 'help("mgcv-package")'.
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
-- Attaching packages -------------------------------------- tidymodels 0.1.3 --
v broom        0.7.9      v rsample      0.1.0 
v dials        0.0.9      v tune         0.1.6 
v infer        0.5.4      v workflows    0.2.3 
v modeldata    0.1.1      v workflowsets 0.1.0 
v parsnip      0.1.5      v yardstick    0.0.8 
v recipes      0.1.16     
-- Conflicts ----------------------------------------- tidymodels_conflicts() --
x 

In [10]:
install.packages("vtreat"); library(vtreat)
install.packages("chron"); library(chron)
install.packages("data.table"); library(data.table)
install.packages("ggfortify"); library(ggfortify)
library(mlbench)
install.packages("wrapr"); library(wrapr)
library(rpart)
library(parsnip)
library(brotools)
install.packages("rsample"); library(rsample)


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: wrapr

Attaching package: 'wrapr'

The following object is masked from 'package:tibble':

    view

The following objects are masked from 'package:tidyr':

    pack, unpack

The following object is masked from 'package:dplyr':

    coalesce


Attaching package: 'vtreat'

The following object is masked from 'package:recipes':

    prepare

The following object is masked from 'package:parsnip':

    fit

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done

Attaching package: 'chron'

The following objects are masked from 'package:lubridate':

    days, hours, minutes, seconds, years

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done

Attaching package: 'data.table'

The following object is masked from 'package:wrapr':

    :=

The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter

ERROR: Error in library(brotools): there is no package called 'brotools'


In [44]:
install.packages("lsr"); library(lsr)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [6]:
#DEFINE HELPER FUNCTION 
load_file <- function(filename) {
    #loads text into dataframe
    return (read.table(filename, header = FALSE, sep = "", stringsAsFactors = FALSE, col.names = c("Date", "Time", "Code", "Value")))
    
}

print(load_file)

function(filename) {
    #loads text into dataframe
    return (read.table(filename, header = FALSE, sep = "", stringsAsFactors = FALSE, col.names = c("Date", "Time", "Code", "Value")))
    
}


In [7]:
#LOADING FILES
D1 <- load_file("data-01")
D1$pid <- c("D1")

head(D1)

Date,Time,Code,Value,pid
04-21-1991,9:09,58,100,D1
04-21-1991,9:09,33,9,D1
04-21-1991,9:09,34,13,D1
04-21-1991,17:08,62,119,D1
04-21-1991,17:08,33,7,D1
04-21-1991,22:51,48,123,D1


In [12]:
D2 <- load_file("data-02")
D2$pid <- c("D2")

In [13]:
D3 <- load_file("data-03")
D3$pid <- c("D3")

In [14]:
D4 <- load_file("data-04")
D4$pid <- c("D4")

In [15]:
D5 <- load_file("data-05")
D5$pid <- c("D5")

In [16]:
D6 <- load_file("data-06")
D6$pid <- c("D6")

In [18]:
D7 <- load_file("data-07")
D7$pid <- c("D7")

In [17]:
D8 <- load_file("data-08")
D8$pid <- c("D8")

In [19]:
D9 <- load_file("data-09")
D9$pid <- c("D9")

In [20]:
D10 <- load_file("data-10")
D10$pid <- c("D10")

In [11]:
#Convert "Date" column from character to date format

#Parse date - "mdy"format of datasets to convert to ISO8601 standard "ymd" format
D1_dates <- D1 %>%
    mutate(date1 = mdy(Date))

head(D1_dates)

Date,Time,Code,Value,pid,date1
04-21-1991,9:09,58,100,D1,1991-04-21
04-21-1991,9:09,33,9,D1,1991-04-21
04-21-1991,9:09,34,13,D1,1991-04-21
04-21-1991,17:08,62,119,D1,1991-04-21
04-21-1991,17:08,33,7,D1,1991-04-21
04-21-1991,22:51,48,123,D1,1991-04-21


In [22]:
#Convert Time column from character to datetime format
D1_dates$datetime = paste(D1_dates$Date, D1_dates$Time)
D1_dates$datetime = mdy_hm(D1_dates$datetime)

glimpse(D1_dates)

Rows: 943
Columns: 7
$ Date     <chr> "04-21-1991", "04-21-1991", "04-21-1991", "04-21-1991", "04-2~
$ Time     <chr> "9:09", "9:09", "9:09", "17:08", "17:08", "22:51", "7:35", "7~
$ Code     <int> 58, 33, 34, 62, 33, 48, 58, 33, 34, 33, 62, 33, 58, 33, 34, 6~
$ Value    <int> 100, 9, 13, 119, 7, 123, 216, 10, 13, 2, 211, 7, 257, 11, 13,~
$ pid      <chr> "D1", "D1", "D1", "D1", "D1", "D1", "D1", "D1", "D1", "D1", "~
$ date1    <date> 1991-04-21, 1991-04-21, 1991-04-21, 1991-04-21, 1991-04-21, ~
$ datetime <dttm> 1991-04-21 09:09:00, 1991-04-21 09:09:00, 1991-04-21 09:09:0~


In [23]:
#Remove redundant character columns ("Date", "Time")
D2_dates <- D1_dates %>%
    select(-Date, -Time)

head(D2_dates)

Code,Value,pid,date1,datetime
58,100,D1,1991-04-21,1991-04-21 09:09:00
33,9,D1,1991-04-21,1991-04-21 09:09:00
34,13,D1,1991-04-21,1991-04-21 09:09:00
62,119,D1,1991-04-21,1991-04-21 17:08:00
33,7,D1,1991-04-21,1991-04-21 17:08:00
48,123,D1,1991-04-21,1991-04-21 22:51:00


In [24]:
#Filter out Insulin datasets

target <- c(33, 34, 35)

Insulin_data <- D2_dates %>%
    filter(Code %in% target) %>%
    rename(Insulin_Code = Code, Insulin_dose = Value)

head(Insulin_data)

Insulin_Code,Insulin_dose,pid,date1,datetime
33,9,D1,1991-04-21,1991-04-21 09:09:00
34,13,D1,1991-04-21,1991-04-21 09:09:00
33,7,D1,1991-04-21,1991-04-21 17:08:00
33,10,D1,1991-04-22,1991-04-22 07:35:00
34,13,D1,1991-04-22,1991-04-22 07:35:00
33,2,D1,1991-04-22,1991-04-22 13:40:00


In [25]:
#Filter out Glucose datasets

target2 <- c(48, 57, 58, 59, 60, 61, 62, 63, 64, 65)

Glucose_data <- D2_dates %>%
    filter(Code %in% target2) %>%
    rename(Glucose_Code = Code) %>%
    rename(Glucose_Value = Value)

head(Glucose_data)

Glucose_Code,Glucose_Value,pid,date1,datetime
58,100,D1,1991-04-21,1991-04-21 09:09:00
62,119,D1,1991-04-21,1991-04-21 17:08:00
48,123,D1,1991-04-21,1991-04-21 22:51:00
58,216,D1,1991-04-22,1991-04-22 07:35:00
62,211,D1,1991-04-22,1991-04-22 16:56:00
58,257,D1,1991-04-23,1991-04-23 07:25:00


In [26]:
#Combine both tables for Insulin and Glucose values

Diabetes_InsGlucose <- Insulin_data %>%
    inner_join(Glucose_data, by = c("date1", "pid", "datetime"))

head(Diabetes_InsGlucose)

Insulin_Code,Insulin_dose,pid,date1,datetime,Glucose_Code,Glucose_Value
33,9,D1,1991-04-21,1991-04-21 09:09:00,58,100
34,13,D1,1991-04-21,1991-04-21 09:09:00,58,100
33,7,D1,1991-04-21,1991-04-21 17:08:00,62,119
33,10,D1,1991-04-22,1991-04-22 07:35:00,58,216
34,13,D1,1991-04-22,1991-04-22 07:35:00,58,216
33,7,D1,1991-04-22,1991-04-22 16:56:00,62,211


In [27]:
#Feature Engineer datetimes into specific months

Diabetes_InsGlucose_dateGroups <- Diabetes_InsGlucose %>%
    mutate(time_period = case_when(datetime >= '1991-04-21' & datetime <= '1991-05-19' ~ "month_1",
                                    datetime >= '1991-05-20' & datetime <= '1991-06-16' ~ "month_2",
                                    datetime >= '1991-06-17' & datetime <= '1991-07-14' ~ "month_3",
                                    datetime >= '1991-07-15' & datetime <= '1991-08-11' ~ "month_4",
                                    datetime >= '1991-08-12' & datetime <= '1991-09-01' ~ "month_5"))

head(Diabetes_InsGlucose_dateGroups)

Insulin_Code,Insulin_dose,pid,date1,datetime,Glucose_Code,Glucose_Value,time_period
33,9,D1,1991-04-21,1991-04-21 09:09:00,58,100,month_1
34,13,D1,1991-04-21,1991-04-21 09:09:00,58,100,month_1
33,7,D1,1991-04-21,1991-04-21 17:08:00,62,119,month_1
33,10,D1,1991-04-22,1991-04-22 07:35:00,58,216,month_1
34,13,D1,1991-04-22,1991-04-22 07:35:00,58,216,month_1
33,7,D1,1991-04-22,1991-04-22 16:56:00,62,211,month_1


In [28]:
#Convert monthly "buckets" to numeric

Diabetes_InsGlucose_months <- Diabetes_InsGlucose_dateGroups %>%
    mutate(
            month_1 = ifelse(time_period == "month_1", 1, 0),
            month_2 = ifelse(time_period == "month_2", 1, 0),
            month_3 = ifelse(time_period == "month_3", 1, 0),
            month_4 = ifelse(time_period == "month_4", 1, 0),
            month_5 = ifelse(time_period == "month_5", 1, 0))

head(Diabetes_InsGlucose_months)

Insulin_Code,Insulin_dose,pid,date1,datetime,Glucose_Code,Glucose_Value,time_period,month_1,month_2,month_3,month_4,month_5
33,9,D1,1991-04-21,1991-04-21 09:09:00,58,100,month_1,1,0,0,0,0
34,13,D1,1991-04-21,1991-04-21 09:09:00,58,100,month_1,1,0,0,0,0
33,7,D1,1991-04-21,1991-04-21 17:08:00,62,119,month_1,1,0,0,0,0
33,10,D1,1991-04-22,1991-04-22 07:35:00,58,216,month_1,1,0,0,0,0
34,13,D1,1991-04-22,1991-04-22 07:35:00,58,216,month_1,1,0,0,0,0
33,7,D1,1991-04-22,1991-04-22 16:56:00,62,211,month_1,1,0,0,0,0


In [29]:
#COHORT ANALYSIS

In [32]:
#Create a training subset of the data

Diab_train <- subset(Diabetes_InsGlucose_months, Diabetes_InsGlucose_months$month_1 == 1)

str(Diab_train)


'data.frame':	86 obs. of  13 variables:
 $ Insulin_Code : int  33 34 33 33 34 33 33 34 33 33 ...
 $ Insulin_dose : int  9 13 7 10 13 7 11 13 7 10 ...
 $ pid          : chr  "D1" "D1" "D1" "D1" ...
 $ date1        : Date, format: "1991-04-21" "1991-04-21" ...
 $ datetime     : POSIXct, format: "1991-04-21 09:09:00" "1991-04-21 09:09:00" ...
 $ Glucose_Code : int  58 58 62 58 58 62 58 58 62 58 ...
 $ Glucose_Value: int  100 100 119 216 216 211 257 257 129 239 ...
 $ time_period  : chr  "month_1" "month_1" "month_1" "month_1" ...
 $ month_1      : num  1 1 1 1 1 1 1 1 1 1 ...
 $ month_2      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ month_3      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ month_4      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ month_5      : num  0 0 0 0 0 0 0 0 0 0 ...


In [33]:
#Calculation of mean Glucose value for month 1 as baseline observed mean

mean_month1 <- Diab_train %>%
    summarise(mean(Glucose_Value))

mean_month1

mean(Glucose_Value)
164.6628


In [37]:
#Manual calculation of a dependent t-test

#Null hypothesis - no difference between mean glucose values between months (i.e. no change in average blood
#glucose levels for the patient between months)

#Alternative hypothesis - significant difference between mean blood glucose values between months

#Significance level set at 0.05

#Define sample size
n <- nrow(Diab_train)
n

#Mean of hte difference scores
mean_diff <- sum(Diab_train$Glucose_Value) / n

#Standard deviation of the difference scores
sd_diff <- sqrt(sum((Diab_train$Glucose_Value - mean_diff) ^2) / (n - 1))

#Observed t-value
t_obs <- mean_diff / (sd_diff / sqrt(n))

#Print observed t-value
t_obs

[1] 86

[1] 19.41309

In [38]:
#Manual Calculation of critical value for comparison with observed t-value (to ascertain if sufficient evidence to 
#reject null hypothesis)
#Degrees of freedom of t-distribution = sample size - 1 (86 - 1 = 85 d.f)

#Two tailed t-test used (expected outcome = significant effect in either POSITIVE or NEGATIVE direction)
#i.e. mean blood glucose levels significantly higher or lower than that for month 1

#Observed t-value = +19.41, so only right side distribution relevant here

#Further, as desired significance level = 0.05, critical value is point on t-distribution where 0.025 of total 
#area of 1 is to right i.e. this point is the 0.975 quantile

#Compute the critical value
t_crit <- qt(0.975, 85)
t_crit

#Print observed t-value 
t_obs

#Compute Cohen's d
cohens_d <- mean_diff / sd_diff
cohens_d

[1] 1.988268

[1] 19.41309

[1] 2.093367

In [39]:
#Interpretation of outputs above

#Observed t-value is significantly larger than the critical value, thus difference is significant at a 
#significance level of 0.05

In [40]:
#Repeating above for month 2
Diab_train_month2 <- subset(Diabetes_InsGlucose_months, Diabetes_InsGlucose_months$month_2 == 1)

In [41]:
#Calculate mean Glucose for month 2
mean_month2 <- Diab_train_month2 %>%
    summarise(mean(Glucose_Value))

mean_month2

mean(Glucose_Value)
161.2


In [42]:
#Calculating t-test statistics for months 1 - 2

t.test(Diab_train$Glucose_Value, Diab_train_month2$Glucose_Value, paired = FALSE)


	Welch Two Sample t-test

data:  Diab_train$Glucose_Value and Diab_train_month2$Glucose_Value
t = 0.30181, df = 175.39, p-value = 0.7632
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -19.18058  26.10616
sample estimates:
mean of x mean of y 
 164.6628  161.2000 


In [45]:
#Computing Cohen's D for months 1 - 2
cohensD(Diab_train$Glucose_Value, Diab_train_month2$Glucose_Value, method = "pooled")

#A Cohen's d of 0.045 implies that the glucose values for the 
#patient changed between months 1 and 2 by 0.045 standard deviations, 
#which is not very large!

[1] 0.04502098

In [46]:
#Repeating above for month 3
Diab_train_month3 <- subset(Diabetes_InsGlucose_months, Diabetes_InsGlucose_months$month_3 == 1)

#Calculate mean Glucose for month 3
mean_month3 <- Diab_train_month3 %>%
    summarise(mean(Glucose_Value))

mean_month3

#Calculating t-test statistics for months 1 - 3

t.test(Diab_train$Glucose_Value, Diab_train_month3$Glucose_Value, paired = FALSE)

mean(Glucose_Value)
168.1158



	Welch Two Sample t-test

data:  Diab_train$Glucose_Value and Diab_train_month3$Glucose_Value
t = -0.31227, df = 170.2, p-value = 0.7552
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -25.28077  18.37477
sample estimates:
mean of x mean of y 
 164.6628  168.1158 


In [47]:
#Computing Cohen's D for months 1 - 3
cohensD(Diab_train$Glucose_Value, Diab_train_month3$Glucose_Value, method = "pooled")

#A Cohen's d of 0.046 means that the glucose values for the 
#patient changed between months 1 and 3 by 0.045 standard deviations, 
#which is not very large!

[1] 0.04677857

In [48]:
#REPEATING ABOVE FOR 4TH MONTH 

Diab_train_month4 <- subset(Diabetes_InsGlucose_months, Diabetes_InsGlucose_months$month_4 == 1)

#Calculate mean Glucose for month 4
mean_month4 <- Diab_train_month4 %>%
    summarise(mean(Glucose_Value))

mean_month4

#Calculating t-test statistics for months 1 - 4

t.test(Diab_train$Glucose_Value, Diab_train_month4$Glucose_Value, paired = FALSE)

#Computing Cohen's D for months 1 - 4
cohensD(Diab_train$Glucose_Value, Diab_train_month4$Glucose_Value, method = "pooled")

#A Cohen's d of 0.07 implies that the glucose values for the 
#patient changed between months 1 and 4 by 0.07 standard deviations, 
#again, not very significant!

mean(Glucose_Value)
159.1951



	Welch Two Sample t-test

data:  Diab_train$Glucose_Value and Diab_train_month4$Glucose_Value
t = 0.48071, df = 164.72, p-value = 0.6314
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -16.99034  27.92568
sample estimates:
mean of x mean of y 
 164.6628  159.1951 


[1] 0.07395521

In [49]:
#REPEATING ABOVE FOR 5TH MONTH 

Diab_train_month5 <- subset(Diabetes_InsGlucose_months, Diabetes_InsGlucose_months$month_5 == 1)

#Calculate mean Glucose for month 5
mean_month5 <- Diab_train_month5 %>%
    summarise(mean(Glucose_Value))

mean_month5

#Calculating t-test statistics for months 1 - 5

t.test(Diab_train$Glucose_Value, Diab_train_month5$Glucose_Value, paired = FALSE)

#Computing Cohen's D for months 1 - 5
cohensD(Diab_train$Glucose_Value, Diab_train_month5$Glucose_Value, method = "pooled")

#A Cohen's d of 0.21 means that the glucose values for the 
#patient changed between months 1 and 5 by 0.21 standard deviations, 
#again, not very significant!

mean(Glucose_Value)
181.3774



	Welch Two Sample t-test

data:  Diab_train$Glucose_Value and Diab_train_month5$Glucose_Value
t = -1.2783, df = 117.17, p-value = 0.2037
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -42.609503   9.180368
sample estimates:
mean of x mean of y 
 164.6628  181.3774 


[1] 0.2188915